In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import metpy.constants as mpconst
from metpy.units import units
import numpy as np
import pint
import xarray as xr

In [ ]:
def read_gdlist(name):
    file = open(name, 'rb')

    for _ in range(6):
        file.readline()

    grid_row = file.readline()
    nlon = int(grid_row[-11:-5].strip())
    nlat = int(grid_row[-4:].strip())
    #print(nlon,nlat)

    gempak_array = np.empty((nlat,nlon))

    for _ in range(2):
        file.readline()

    scale_row = file.readline()
    scale_factor = int(scale_row[19:].strip())
    #print(scale_factor)

    for _ in range(92):
        file.readline()

    col = 0
    row = 0
    for line in file.readlines():
        if (line[:13].strip() != b'') & (col >= nlon):
            col = 0
            row+=1
        for count in [0,1,2,3,4,5,6,7]:
            gempak_array[row,col] = line[8+count*9:17+count*9]
            col+=1

    file.close()
    
    return (gempak_array, -scale_factor)

In [ ]:
def error_stats(metpy_data, gempak_data):
    ignore = (gempak_data != -9999.0) * ~np.isnan(metpy_data)
    
    if isinstance(metpy_data.data, pint.Quantity):
        print(f'Is Xarray with Quantity? True')
        mdata = metpy_data.values[ignore]
    elif isinstance(metpy_data, pint.Quantity):
        print(f'Is Metpy Unit Array Quantity? True')
        mdata = metpy_data.m[ignore]
    else:
        print(f'Is Quantity? False')
        try:
            mdata = metpy_data.values[ignore]
        except:
            mdata = metpy_data[ignore]
    gempak_data = gempak_data[ignore]
    
    print()
    print('Mean Comparison')
    print(f'  Mean Values (MetPy): {mdata.mean()}')
    print(f'  Mean Values (GEMPAK): {gempak_data.mean()}')
    print()
    print('Max Comparison')
    print(f'  Max Values (MetPy): {mdata.max()}')
    print(f'  Max Values (GEMPAK): {gempak_data.max()}')
    print()
    print('Min Comparison')
    print(f'  Min Values (MetPy): {mdata.min()}')
    print(f'  Min Values (GEMPAK): {gempak_data.min()}')

    print()
    print('Difference Array')
    diff = mdata - gempak_data
    print(diff)
    print()
    print('Various Statistical Analyses')
    print(f'  Average Absolute Difference: {np.nanmean(np.abs(diff))}')
    print(f'  RMS Error: {np.sqrt(np.nansum(diff**2))/len(diff.ravel())}')
    print(f'  Standard Deviation of Difference: {np.nanstd(diff)}')
    print(f'  Max Diff: {np.max(diff)}')
    print(f'  Min Diff: {np.min(diff)}')
    print(f'  Correlation: {np.corrcoef(mdata.ravel(), gempak_data.ravel())[0][1]}')
    print(f'  Relative Magnitude Difference: {np.nanmean(np.abs(diff))/np.nanmax(mdata)}')
    print()

In [ ]:
gfs_data = xr.open_dataset('gfs_test_data.nc').metpy.parse_cf()

In [ ]:
# Ageostrophic Wind

gempak_out_var_u, scale_u = read_gdlist('gempak_out_files/gempak_vector_function_u_comp_ageo.out')
gempak_out_var_v, scale_v = read_gdlist('gempak_out_files/gempak_vector_function_v_comp_ageo.out')

out_var_u = gempak_out_var_u * 10**scale_u
out_var_v = gempak_out_var_v * 10**scale_v

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_lat = gfs_data.lat

uageo, vageo = mpcalc.ageostrophic_wind(data_var, data_var_u, data_var_v,
                                        f=mpcalc.coriolis_parameter(data_var_lat.values[:, None] * units.degree))

# Subset to NH and not too tropical
NH_lat = int(360/2)-9
NH_lon = int(720/2)-9
print('Ageostrophic Wind')
print('U-component')
error_stats(uageo[3:NH_lat, 3:NH_lon], out_var_u[3:NH_lat, 3:NH_lon])
print()
print('V-component')
error_stats(vageo[3:NH_lat, 3:NH_lon], out_var_v[3:NH_lat, 3:NH_lon])

# Plot Check
fig = plt.figure(1, figsize=(10, 10))
ax = plt.subplot(111)

wind_slice = slice(60, 170, 10)
ax.quiver(uageo[wind_slice, wind_slice], vageo[wind_slice, wind_slice], alpha=0.75)
ax.quiver(out_var_u[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice], color='red', alpha=0.5)
plt.title('Test Ageostrophic Output GEMPAK(red), MetPy (black)')
plt.show()

In [ ]:
# Geostrophic Wind

gempak_out_var_u, scale_u = read_gdlist('gempak_out_files/gempak_vector_function_u_comp_geo.out')
gempak_out_var_v, scale_v = read_gdlist('gempak_out_files/gempak_vector_function_v_comp_geo.out')

out_var_u = gempak_out_var_u * 10**scale_u
out_var_v = gempak_out_var_v * 10**scale_v

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_lat = gfs_data.lat

ugeo, vgeo = mpcalc.geostrophic_wind(data_var, f=mpcalc.coriolis_parameter(data_var_lat.values[:, None] * units.degree))

# Subset to NH and not too tropical
NH_lat = int(360/2)-9
NH_lon = int(720/2)-9
print('Geostrophic Wind')
print('U-component')
error_stats(ugeo[1:NH_lat, 1:NH_lon], out_var_u[1:NH_lat, 1:NH_lon])
print()
print('V-component')
error_stats(vgeo[1:NH_lat, 1:NH_lon], out_var_v[1:NH_lat, 1:NH_lon])

# Plot Check
fig = plt.figure(1, figsize=(10, 10))
ax = plt.subplot(111)

wind_slice = slice(60, 170, 10)
ax.quiver(ugeo[wind_slice, wind_slice], vgeo[wind_slice, wind_slice], alpha=0.75)
ax.quiver(out_var_u[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice], color='red', alpha=0.5)
plt.title('Test Geostrophic Output GEMPAK(red), MetPy (black)')
plt.show()

In [ ]:
# Gradient of Scalar

gempak_out_var_u, scale_u = read_gdlist('gempak_out_files/gempak_vector_function_u_comp_grad.out')
gempak_out_var_v, scale_v = read_gdlist('gempak_out_files/gempak_vector_function_v_comp_grad.out')

out_var_u = gempak_out_var_u * 10**scale_u
out_var_v = gempak_out_var_v * 10**scale_v

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

gradient_vcomp, gradient_ucomp = mpcalc.gradient(data_var)

print('Gradient of Scalar')
print('U-component')
error_stats(gradient_ucomp[1:-1, 1:-1], out_var_u[1:-1, 1:-1])
print()
print('V-component')
error_stats(gradient_vcomp[1:-1, 1:-1], out_var_v[1:-1, 1:-1])

# Plot Check
fig = plt.figure(1, figsize=(10, 10))
ax = plt.subplot(111)

wind_slice = slice(60, 170, 10)
ax.quiver(gradient_ucomp[wind_slice, wind_slice], gradient_vcomp[wind_slice, wind_slice], alpha=0.75)
ax.quiver(out_var_u[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice], color='red', alpha=0.5)
plt.title('Test Gradient Output GEMPAK(red), MetPy (black)')
plt.show()

In [ ]:
# Inertial Advective

gempak_out_var_u, scale_u = read_gdlist('gempak_out_files/gempak_vector_function_u_comp_inad.out')
gempak_out_var_v, scale_v = read_gdlist('gempak_out_files/gempak_vector_function_v_comp_inad.out')

out_var_u = gempak_out_var_u * 10**scale_u
out_var_v = gempak_out_var_v * 10**scale_v

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_lat = gfs_data.lat

ugeo, vgeo = mpcalc.geostrophic_wind(data_var, f=mpcalc.coriolis_parameter(data_var_lat.values[:, None] * units.degree))

v_inad, u_inad = mpcalc.inertial_advective_wind(data_var_u, data_var_v, ugeo, vgeo,
                                                latitude=data_var_lat.values[:, None] * units.degree)

u_inad *= mpcalc.coriolis_parameter(data_var_lat.values[:, None] * units.degree)
v_inad *= -mpcalc.coriolis_parameter(data_var_lat.values[:, None] * units.degree)

# Subset to NH and not too tropical
NH_lat = int(360/2)-10
NH_lon = -1
print('Inertial-Advective Wind')
print('U-component')
error_stats(u_inad[2:NH_lat, 1:NH_lon], out_var_u[2:NH_lat, 1:NH_lon])
print()
print('V-component')
error_stats(v_inad[2:NH_lat, 1:NH_lon], out_var_v[2:NH_lat, 1:NH_lon])

# Plot Check
fig = plt.figure(1, figsize=(10, 10))
ax = plt.subplot(111)

wind_slice = slice(60, 170, 10)
ax.quiver(u_inad[wind_slice, wind_slice], v_inad[wind_slice, wind_slice], alpha=0.75)
ax.quiver(out_var_u[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice], color='red', alpha=0.5)
plt.title('Test Inertial-Advective Output GEMPAK(red), MetPy (black)')
plt.show()

In [ ]:
# QVectors

gempak_out_var_u, scale_u = read_gdlist('gempak_out_files/gempak_vector_function_u_comp_qvec.out')
gempak_out_var_v, scale_v = read_gdlist('gempak_out_files/gempak_vector_function_v_comp_qvec.out')

out_var_u = gempak_out_var_u * 10**scale_u
out_var_v = gempak_out_var_v * 10**scale_v

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000)
data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000)
data_var_lat = gfs_data.lat

u_qvect, v_qvect = mpcalc.q_vector(data_var_u, data_var_v,
                                   mpcalc.potential_temperature(850 * units.hPa, data_var), 850 * units.hPa)

u_qvect = (u_qvect.metpy.unit_array / mpconst.Rd * 850 * units.hPa).to_base_units()
v_qvect = (v_qvect.metpy.unit_array / mpconst.Rd * 850 * units.hPa).to_base_units()

wind_slice = slice(3, -3, None)
print('Q-Vectors')
print('U-component')
error_stats(u_qvect[wind_slice, wind_slice], out_var_u[wind_slice, wind_slice])
print()
print('V-component')
error_stats(v_qvect[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice])

# Plot Check
fig = plt.figure(3, figsize=(10, 10))
ax = plt.subplot(111)

wind_slice = slice(60, 170, 10)
ax.quiver(u_qvect[wind_slice, wind_slice], v_qvect[wind_slice, wind_slice], pivot='mid', scale=1e-8)
ax.quiver(out_var_u[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice], color='red', pivot='mid', scale=1e-8, alpha=0.5)
plt.title('Test Q-Vector Output GEMPAK(red), MetPy (black)')
plt.show()

In [ ]:
# KNTV

gempak_out_var_u, scale_u = read_gdlist('gempak_out_files/gempak_vector_function_u_comp_kntv.out')
gempak_out_var_v, scale_v = read_gdlist('gempak_out_files/gempak_vector_function_v_comp_kntv.out')

out_var_u = gempak_out_var_u * 10**scale_u
out_var_v = gempak_out_var_v * 10**scale_v

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

data_var_u_kt = data_var_u.metpy.convert_units('knot')
data_var_v_kt = data_var_v.metpy.convert_units('knot')

data_var_u_kt = data_var_u * 1.9425
data_var_v_kt = data_var_v * 1.9425

wind_slice = slice(3, -3, None)
print('Knot Unit Conversion')
print('U-component')
error_stats(data_var_u_kt[wind_slice, wind_slice], out_var_u[wind_slice, wind_slice])
print()
print('V-component')
error_stats(data_var_v_kt[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice])

# Plot Check
fig = plt.figure(3, figsize=(10, 10))
ax = plt.subplot(111)

wind_slice = slice(60, 170, 10)
ax.quiver(data_var_u_kt[wind_slice, wind_slice], data_var_v_kt[wind_slice, wind_slice], pivot='mid', alpha=0.75)
ax.quiver(out_var_u[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice], color='red', pivot='mid', alpha=0.5)
plt.title('Test Knot Unit Conversion Output GEMPAK(red), MetPy (black)')
plt.show()

In [ ]:
# SM5V

gempak_out_var_u, scale_u = read_gdlist('gempak_out_files/gempak_vector_function_u_comp_sm5v.out')
gempak_out_var_v, scale_v = read_gdlist('gempak_out_files/gempak_vector_function_v_comp_sm5v.out')

out_var_u = gempak_out_var_u * 10**scale_u
out_var_v = gempak_out_var_v * 10**scale_v

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

data_var_u_sm5v = mpcalc.smooth_n_point(data_var_u, 5, 1)
data_var_v_sm5v = mpcalc.smooth_n_point(data_var_v, 5, 1)


wind_slice = slice(3, -3, None)
print('5-point Smoother')
print('U-component')
error_stats(data_var_u_sm5v[wind_slice, wind_slice], out_var_u[wind_slice, wind_slice])
print()
print('V-component')
error_stats(data_var_v_sm5v[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice])

# Plot Check
fig = plt.figure(3, figsize=(10, 10))
ax = plt.subplot(111)

wind_slice = slice(60, 170, 10)
ax.quiver(data_var_u_sm5v[wind_slice, wind_slice], data_var_v_sm5v[wind_slice, wind_slice], pivot='mid', alpha=0.75)
ax.quiver(out_var_u[wind_slice, wind_slice], out_var_v[wind_slice, wind_slice], color='red', pivot='mid', alpha=0.5)
plt.title('Test 5-pt Smoother Output GEMPAK(red), MetPy (black)')
plt.show()